In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [2]:
standings_url = "https://www.baseball-reference.com/"
data = requests.get(standings_url)
soup = BeautifulSoup(data.text)
standings_table = soup.select('table#standings_AL')[0]


In [6]:
links = standings_table.find_all('a')
links = [l.get("href") for l in links] 
team_urls = [f"https://www.baseball-reference.com{l}" for l in links] 

all_matches = []

for team_url in team_urls:
    data = requests.get(team_url) 
    soup = BeautifulSoup(data.text)
    links = soup.find_all('a') # find all links on page for the club
    links = [l.get("href") for l in links] # get reference urls for those links
    links = [l for l in links if l and (('schedule-scores' in l))]
    links = [*set(links)][0]
    sch_url = f"https://www.baseball-reference.com{links}"
    data = requests.get(sch_url) 
    sch = pd.read_html(data.text, match='Team Game-by-Game Schedule')[0]
    sch = sch.drop(sch[sch['Gm#']=='Gm#'].index)
    sch = sch.drop(sch[sch['Streak'].isna()].index)
    sch = sch[['Gm#', 'Date', 'Tm', 'Unnamed: 4', 'Opp', 'W/L', 'R',
       'RA', 'Inn', 'W-L', 'Rank', 'GB', 'Win', 'Loss', 'Save', 'Time', 'D/N',
       'Attendance', 'Streak']]

    all_matches.append(sch)
    time.sleep(1)




In [18]:
match_df

,Gm#,Date,Tm,Unnamed: 4,Opp,W/L,R,RA,Inn,W-L,Rank,GB,Win,Loss,Save,Time,D/N,Attendance,Streak
1171,23,"Wednesday, May 4",DET,NaN,PIT,L,2,7,NaN,8-15,4,6.0,Thompson,Hutchison,Bednar,3:07,D,15150,-
662,24,"Wednesday, May 4",CLE,NaN,SDP,W-wo,6,5,10,11-13,2,3.5,Sandlin,Lamet,NaN,3:46,D,10244,+
661,23,"Wednesday, May 4",CLE,NaN,SDP,L,4,5,NaN,10-13,2,3.5,Crismatt,Plesac,Rogers,3:15,D,NaN,-
1170,22,"Wednesday, May 4",DET,NaN,PIT,W,3,2,NaN,8-14,4,6.0,Barnes,Wilson,Soto,2:41,D,NaN,+
1814,25,"Wednesday, May 4",OAK,NaN,TBR,L,0,3,NaN,10-15,5,5.5,Feyereisen,Jackson,Kittredge,2:39,D,4838,------
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1283,7,"Friday, Apr 15",HOU,@,SEA,L,1,11,NaN,4-3,1,up 0.5,Gonzales,Odorizzi,NaN,3:04,N,45023,--
1668,8,"Friday, Apr 15",LAA,@,TEX,W,9,6,NaN,4-4,2,0.5,Warren,Allard,Iglesias,3:32,N,28723,+
263,8,"Friday, Apr 15",TOR,NaN,OAK,W,4,1,NaN,5-3,1,up 1.0,Cimber,Jefferies,Romano,3:00,N,35415,+
1156,8,"Friday, Apr 15",DET,@,KCR,W,2,1,NaN,4-4,3,1.5,Barnes,Keller,Fulmer,2:18,N,16720,++


In [38]:
match_df = pd.concat(all_matches).reset_index(drop=True)
match_df = match_df.sort_values('Date', ascending=False)
match_df['Date'] = match_df['Date'].str.split('(').str[0]
match_df['Date'] = match_df['Date'].str.replace(' ', '')
match_df['Date'] = match_df['Date'].str.replace(',', '')

match_df['Date'] = pd.to_datetime(match_df['Date'], format='%A%b%d')
match_df['Date']  = match_df['Date'].dt.strftime('%m/%d/2022')

match_df

,Gm#,Date,Tm,Unnamed: 4,Opp,W/L,R,RA,Inn,W-L,Rank,GB,Win,Loss,Save,Time,D/N,Attendance,Streak
1171,23,05/04/2022,DET,NaN,PIT,L,2,7,NaN,8-15,4,6.0,Thompson,Hutchison,Bednar,3:07,D,15150,-
662,24,05/04/2022,CLE,NaN,SDP,W-wo,6,5,10,11-13,2,3.5,Sandlin,Lamet,NaN,3:46,D,10244,+
661,23,05/04/2022,CLE,NaN,SDP,L,4,5,NaN,10-13,2,3.5,Crismatt,Plesac,Rogers,3:15,D,NaN,-
1170,22,05/04/2022,DET,NaN,PIT,W,3,2,NaN,8-14,4,6.0,Barnes,Wilson,Soto,2:41,D,NaN,+
1814,25,05/04/2022,OAK,NaN,TBR,L,0,3,NaN,10-15,5,5.5,Feyereisen,Jackson,Kittredge,2:39,D,4838,------
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1283,7,04/15/2022,HOU,@,SEA,L,1,11,NaN,4-3,1,up 0.5,Gonzales,Odorizzi,NaN,3:04,N,45023,--
1668,8,04/15/2022,LAA,@,TEX,W,9,6,NaN,4-4,2,0.5,Warren,Allard,Iglesias,3:32,N,28723,+
263,8,04/15/2022,TOR,NaN,OAK,W,4,1,NaN,5-3,1,up 1.0,Cimber,Jefferies,Romano,3:00,N,35415,+
1156,8,04/15/2022,DET,@,KCR,W,2,1,NaN,4-4,3,1.5,Barnes,Keller,Fulmer,2:18,N,16720,++
